# **IP3_Run3**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 7.4 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """You are the person in charge of packing in a large company. Your job is to skillfully pack items of various weights in a box with predetermined capacity. The objective is to use as few boxes as possible. There is a set of items and a set of boxes with an upper bound on the amount of boxes. Each of the items has a known weight. Each item is packed into one box and items are packed into the chosen boxes. Please formulate a mathematical optimization model for this problem.  """

In [5]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 3
)


In [6]:
print(response1.choices[0].message.content)

1. Decision Variables:
   - Binary variable x[i, j] : 1 if item i is placed in box j, 0 otherwise.
   - Binary variable y[j]: 1 if box j is used, 0 otherwise.

2. Parameters:
   - W[i] : Weight of item i.
   - C[j] : Capacity of box j.
   - B: Upper bound on the amount of boxes.   
   - N : Number of items. 
   - M : Number of boxes.
  
3. Index sets:
   - i ∈ N : Index set of items.
   - j ∈ M : Index set of boxes.


### **Generate Pyomo Code**

In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [ ]:
print(response2.choices[0].message.content)

Objective Function:

Minimize ∑_(j=1)^M Y_j


In [8]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [9]:
print(response3.choices[0].message.content)

Constraints:

1. Each item should be placed in one and only one box:
∑x[i, j] for all j ∈ M = 1, for all i ∈ N

2. The total weight in each box should not exceed its capacity:
∑W[i]*x[i, j] for all i ∈ N ≤ C[j]*y[j], for all j ∈ M 

3. The number of boxes used should not exceed the upper bound:
∑y[j] for all j ∈ M ≤ B

4. Decision variables are Binary:
x[i, j], y[j] ∈ {0, 1}, for all i ∈ N and for all j ∈ M


In [10]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [11]:
Markdown(response4.choices[0].message.content)

Here is a Python Pyomo code that models the problem described above:

```python
from pyomo.environ import *

# sample data
N = 5  # number of items
M = 3  # number of boxes
B = 2  # upper bound on the number of boxes
W = {1: 1, 2: 2, 3: 3, 4: 4, 5: 5}  # weight of each item
C = {1: 10, 2: 10, 3: 10}  # capacity of each box

# create a model
model = ConcreteModel()

# sets
model.I = RangeSet(1, N)
model.J = RangeSet(1, M)

# variables
model.x = Var(model.I, model.J, within=Binary)
model.y = Var(model.J, within=Binary)

# objective
model.obj = Objective(expr=sum(model.y[j] for j in model.J), sense=minimize)

# constraints
model.single_box = Constraint(model.I, rule=lambda model, i: sum(model.x[i, j] for j in model.J) == 1)
model.capacity = Constraint(model.J, rule=lambda model, j: sum(W[i]*model.x[i, j] for i in model.I) <= C[j]*model.y[j])
model.box_limit = Constraint(rule=lambda model: sum(model.y[j] for j in model.J) <= B)

# solve
solver = SolverFactory('glpk')
solver.solve(model)

# print the output
for i in model.I:
    for j in model.J:
        if model.x[i, j].value == 1:
            print(f"Item {i} is in box {j}")
for j in model.J:
    if model.y[j].value == 1:
        print(f"Box {j} is used")
```

In this code, we adapted your mathematical model into a Pyomo model. Note that the constraints are written as Python functions. This code uses the GLPK solver to solve the problem. It prints out the results indicating which items have been placed in which boxes, and which boxes have been used. Sample data are used for the number of items and boxes, weights of the items, capacities of the boxes, and the box limit.

### **Run the code Generated by GPT4**

In [12]:
from pyomo.environ import *

# sample data
N = 5  # number of items
M = 3  # number of boxes
B = 2  # upper bound on the number of boxes
W = {1: 1, 2: 2, 3: 3, 4: 4, 5: 5}  # weight of each item
C = {1: 10, 2: 10, 3: 10}  # capacity of each box

# create a model
model = ConcreteModel()

# sets
model.I = RangeSet(1, N)
model.J = RangeSet(1, M)

# variables
model.x = Var(model.I, model.J, within=Binary)
model.y = Var(model.J, within=Binary)

# objective
model.obj = Objective(expr=sum(model.y[j] for j in model.J), sense=minimize)

# constraints
model.single_box = Constraint(model.I, rule=lambda model, i: sum(model.x[i, j] for j in model.J) == 1)
model.capacity = Constraint(model.J, rule=lambda model, j: sum(W[i]*model.x[i, j] for i in model.I) <= C[j]*model.y[j])
model.box_limit = Constraint(rule=lambda model: sum(model.y[j] for j in model.J) <= B)

# solve
solver = SolverFactory('glpk')
solver.solve(model)

# print the output
for i in model.I:
    for j in model.J:
        if model.x[i, j].value == 1:
            print(f"Item {i} is in box {j}")
for j in model.J:
    if model.y[j].value == 1:
        print(f"Box {j} is used")

Item 1 is in box 3
Item 2 is in box 3
Item 3 is in box 3
Item 4 is in box 3
Item 5 is in box 1
Box 1 is used
Box 3 is used


In [20]:
from pyomo.environ import *

# sample data
N = 24  # number of items
M = 15  # number of boxes
B = 15  # upper bound on the number of boxes
W = {1: 2,2: 2,3: 2,4: 2,5: 3,6: 3,7:4,8: 4,9: 4,10: 4,11: 4,12:4,13: 5,14:5,15: 5,16: 5,17: 5,18: 5,19: 6,20:6,21: 7,22: 7,23: 8, 24:8} # Weight of each item
  # weight of each item
C = 9  # capacity of each box

# create a model
model = ConcreteModel()

# sets
model.I = RangeSet(1, N)
model.J = RangeSet(1, M)

# variables
model.x = Var(model.I, model.J, within=Binary)
model.y = Var(model.J, within=Binary)

# objective
model.obj = Objective(expr=sum(model.y[j] for j in model.J), sense=minimize)

# constraints
model.single_box = Constraint(model.I, rule=lambda model, i: sum(model.x[i, j] for j in model.J) == 1)
model.capacity = Constraint(model.J, rule=lambda model, j: sum(W[i]*model.x[i, j] for i in model.I) <= C*model.y[j])
model.box_limit = Constraint(rule=lambda model: sum(model.y[j] for j in model.J) <= B)

# solve
solver = SolverFactory('glpk')
solver.solve(model)

# print the output
for i in model.I:
    for j in model.J:
        if model.x[i, j].value == 1:
            print(f"Item {i} is in box {j}")
for j in model.J:
    if model.y[j].value == 1:
        print(f"Box {j} is used")
print(model.obj())

Item 1 is in box 3
Item 2 is in box 5
Item 3 is in box 2
Item 4 is in box 4
Item 5 is in box 14
Item 6 is in box 6
Item 7 is in box 8
Item 8 is in box 7
Item 9 is in box 9
Item 10 is in box 10
Item 11 is in box 10
Item 12 is in box 11
Item 13 is in box 8
Item 14 is in box 7
Item 15 is in box 9
Item 16 is in box 4
Item 17 is in box 6
Item 18 is in box 11
Item 19 is in box 14
Item 20 is in box 5
Item 21 is in box 2
Item 22 is in box 3
Item 23 is in box 1
Item 24 is in box 12
Box 1 is used
Box 2 is used
Box 3 is used
Box 4 is used
Box 5 is used
Box 6 is used
Box 7 is used
Box 8 is used
Box 9 is used
Box 10 is used
Box 11 is used
Box 12 is used
Box 14 is used
13.0


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**